In [ ]:
import setup
import essentia as es
import essentia.standard as es_mode

In [ ]:
def SuperFluxSegmentation(audio, combine=100, ratioThreshold=10, threshold=0.05):

  algo = es_mode.SuperFluxExtractor(combine=combine, 
                                ratioThreshold=ratioThreshold,
                                threshold=threshold)
  onsets = algo(audio)
  end = es_mode.Duration()(audio)
  end_times = es.array(np.append(onsets, end))
  start_times = es.array(np.append([0], onsets))

  segments = es_mode.Slicer(endTimes=end_times,
                    startTimes=start_times,
                    timeUnits="seconds")(audio)

  return segments, onsets

In [ ]:
def OnsetsSegmentation(audio, frame_size=1024, frame_hop=512,
                       windowing_type='hann', onsets_method='hfc'):
  
  #declaração dos algoritmos que serão usados
  spec = es_mode.Spectrum()
  fft = es_mode.FFT()
  c2p = es_mode.CartesianToPolar()
  od1 = es_mode.OnsetDetection(method=onsets_method)
  w = es_mode.Windowing(type=windowing_type)
  pool = es.Pool()

  #Função que será executada a cada frame
  def F(n):
    spectrum = spec(w(n))
    mag, phase, = c2p(fft(w(n)))
    pool.add('features.spectrum', spectrum)
    pool.add('features.', phase)
    pool.add('features.onsetdetection', od1(spectrum ,phase))

  #define a função contínua de onsets para cada frame
  qtdFrames = inFrames( audio=audio,algorithm=F, frameSize=frame_size, hopSize=frame_hop)
  #print("Quantidade de frames: ", qtdFrames)

  audio_duration = es_mode.Duration()(audio)
  frame_rate = qtdFrames/audio_duration
  os = es_mode.Onsets(frameRate=frame_rate)

  #matriz de algoritmos de detecção de onset executados
  onset_detection_matrix = es.array([pool['features.onsetdetection']])

  #segundo parâmetro é o vetor de pesos para cada detecção de onset
  onsets = os(onset_detection_matrix, [1])

  end_times = es.array(np.append(onsets, audio_duration))
  start_times = es.array(np.append([0], onsets))
  segments = es_mode.Slicer(endTimes=end_times,
                    startTimes=start_times,
                    timeUnits="seconds")(audio)
  return segments, onsets
  